# 📊 Exploration et Analyse des Données - Seattle Building Energy## 🎯 ObjectifCe notebook réalise l'**analyse exploratoire (EDA)** des données de consommation énergétique des bâtiments de Seattle.**Objectifs spécifiques :**- Explorer la structure et la qualité des données- Identifier et traiter les valeurs aberrantes (outliers)- Créer de nouvelles features pertinentes (feature engineering)- Préparer les données pour la modélisation**Dataset** : [2016 Building Energy Benchmarking](https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy)

---## 1. 📦 Imports et Configuration

In [ ]:
# =============================================================================# IMPORTS# =============================================================================# Data manipulationimport pandas as pdimport numpy as np# Visualizationimport matplotlib.pyplot as pltimport seaborn as sns# Statisticsfrom scipy import stats# Machine Learning - Preprocessingfrom sklearn.preprocessing import StandardScalerfrom sklearn.experimental import enable_iterative_imputerfrom sklearn.impute import IterativeImputer# Configurationpd.set_option('display.max_columns', 50)plt.style.use('seaborn-v0_8-whitegrid')sns.set_palette("husl")# Suppress warnings for cleaner outputimport warningswarnings.filterwarnings('ignore', category=FutureWarning)print("✅ Imports chargés avec succès")

---## 2. 📁 Chargement des Données

In [ ]:
# =============================================================================# CHARGEMENT DES DONNÉES# =============================================================================# Chemins des fichiersDATA_PATH = '../data/2016_Building_Energy_Benchmarking.csv'OUTPUT_PATH = '../data/data_cleaned.csv'# Chargementraw_data = pd.read_csv(DATA_PATH)# Aperçu initialprint(f"📊 Dimensions du dataset: {raw_data.shape[0]} lignes × {raw_data.shape[1]} colonnes")print(f"📁 Mémoire utilisée: {raw_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")raw_data.head()

In [ ]:
# Informations sur les types de donnéesraw_data.info()

---## 3. 🔍 Exploration Initiale### 3.1 Types de Bâtiments

In [ ]:
# =============================================================================# ANALYSE DES TYPES DE BÂTIMENTS# =============================================================================print("📊 Distribution des types de bâtiments:\n")print(raw_data['BuildingType'].value_counts(dropna=False))# Visualisationfig, ax = plt.subplots(figsize=(10, 5))raw_data['BuildingType'].value_counts().plot(kind='barh', ax=ax, color='steelblue')ax.set_xlabel('Nombre de bâtiments')ax.set_title('Distribution des Types de Bâtiments')plt.tight_layout()plt.show()

### 3.2 Filtrage des DonnéesNous nous concentrons sur les **bâtiments non résidentiels** uniquement, en excluant les "Multifamily".

In [ ]:
# =============================================================================# FILTRAGE - BÂTIMENTS NON RÉSIDENTIELS UNIQUEMENT# =============================================================================# Copie de travaildf = raw_data.copy()# Exclusion des bâtiments multifamiliauxmask_multifamily = df['BuildingType'].str.startswith('Multifamily', na=False)df = df[~mask_multifamily].copy()# Définir l'indexdf.set_index('OSEBuildingID', inplace=True)print(f"✅ Bâtiments restants après filtrage: {len(df)}")print(f"   (Exclus: {mask_multifamily.sum()} bâtiments multifamiliaux)")

---## 4. 🛠️ Feature EngineeringCréation de nouvelles variables potentiellement pertinentes pour la modélisation.

In [ ]:
# =============================================================================# CRÉATION DE NOUVELLES FEATURES# =============================================================================# 1. Âge du bâtimentdf['Age'] = df['DataYear'] - df['YearBuilt']# 2. Ratios de surfacedf['PropertyGFAParking_Pct'] = (df['PropertyGFAParking'] / df['PropertyGFATotal'] * 100).fillna(0)df['PropertyGFABuilding_Pct'] = (df['PropertyGFABuilding(s)'] / df['PropertyGFATotal'] * 100).fillna(0)# 3. Pourcentages d'utilisation énergétique par source# (Éviter division par zéro)energy_total = df['SiteEnergyUse(kBtu)'].replace(0, np.nan)df['SteamUse_Pct'] = (df['SteamUse(kBtu)'] / energy_total * 100)df['Electricity_Pct'] = (df['Electricity(kBtu)'] / energy_total * 100)df['NaturalGas_Pct'] = (df['NaturalGas(kBtu)'] / energy_total * 100)print("✅ Nouvelles features créées:")print("   - Age (âge du bâtiment)")print("   - PropertyGFAParking_Pct (% surface parking)")print("   - PropertyGFABuilding_Pct (% surface bâtiment)")print("   - SteamUse_Pct, Electricity_Pct, NaturalGas_Pct (% par source d'énergie)")

---## 5. 🧹 Nettoyage des Données### 5.1 Suppression des Outliers Flaggés

In [ ]:
# =============================================================================# TRAITEMENT DES OUTLIERS# =============================================================================# Suppression des outliers déjà identifiés dans le datasetoutliers_count = df['Outlier'].notna().sum()df = df[df['Outlier'].isna()].copy()print(f"✅ Outliers supprimés: {outliers_count}")print(f"   Bâtiments restants: {len(df)}")

### 5.2 Suppression des Valeurs Aberrantes

In [ ]:
# =============================================================================# SUPPRESSION DES VALEURS NÉGATIVES# =============================================================================# Électricité négative n'a pas de sensnegative_electricity = (df['Electricity(kBtu)'] < 0).sum()df = df[df['Electricity(kBtu)'] >= 0].copy()# Remplacer les infinis par NaNdf.replace([np.inf, -np.inf], np.nan, inplace=True)print(f"✅ Lignes avec électricité négative supprimées: {negative_electricity}")

### 5.3 Sélection des Colonnes Pertinentes

In [ ]:
# =============================================================================# SÉLECTION DES FEATURES# =============================================================================# Colonnes à supprimer (métadonnées, redondantes, ou non pertinentes)columns_to_drop = [    # Métadonnées    'DataYear', 'BuildingType', 'PropertyName', 'Address', 'City', 'State',     'ZipCode', 'TaxParcelIdentificationNumber', 'Neighborhood',     'Latitude', 'Longitude', 'Comments', 'DefaultData', 'ComplianceStatus', 'Outlier',        # Variables redondantes ou utilisées pour créer d'autres features    'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',     'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',    'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',    'YearsENERGYSTARCertified', 'YearBuilt',        # Unités alternatives (garder kBtu uniquement)    'Electricity(kWh)', 'NaturalGas(therms)',        # Variables utilisées pour calculer les pourcentages    'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',    'PropertyGFAParking', 'PropertyGFABuilding(s)']# Supprimer les colonnes existantes seulementcolumns_to_drop = [col for col in columns_to_drop if col in df.columns]df.drop(columns=columns_to_drop, inplace=True)print(f"✅ Colonnes restantes: {df.shape[1]}")print(f"   {list(df.columns)}")

---## 6. 📊 Analyse des Types de Propriété

In [ ]:
# =============================================================================# CONSOLIDATION DES TYPES DE PROPRIÉTÉ# =============================================================================print("Distribution actuelle des types de propriété:\n")print(df['PrimaryPropertyType'].value_counts())# Regrouper les catégories avec peu d'observations dans 'Other'rare_categories = [    'Residence Hall', 'Senior Care Community', 'Refrigerated Warehouse',    'Restaurant', 'Hospital', 'Laboratory', 'Office', 'Low-Rise Multifamily']for cat in rare_categories:    if cat == 'Low-Rise Multifamily':        # Supprimer complètement (hors scope)        df = df[df['PrimaryPropertyType'] != cat].copy()    else:        # Regrouper dans 'Other'        df.loc[df['PrimaryPropertyType'] == cat, 'PrimaryPropertyType'] = 'Other'print("\n✅ Distribution après consolidation:\n")print(df['PrimaryPropertyType'].value_counts())

---## 7. 🔢 Encodage des Variables Catégorielles

In [ ]:
# =============================================================================# ONE-HOT ENCODING# =============================================================================# Encodage des types de propriétédf = pd.get_dummies(df, columns=['PrimaryPropertyType'], prefix='PropType')# Encodage des districtsdf = pd.get_dummies(df, columns=['CouncilDistrictCode'], prefix='District')# Feature indicatrice pour ENERGYSTARScore manquantdf['ENERGYSTARScore_Missing'] = df['ENERGYSTARScore'].isna().astype(int)print(f"✅ Encodage terminé")print(f"   Dimensions finales: {df.shape}")

---## 8. 📈 Visualisations Exploratoires### 8.1 Distribution des Variables Numériques

In [ ]:
# =============================================================================# HISTOGRAMMES DES VARIABLES CLÉS# =============================================================================numeric_cols = [    'NumberofFloors', 'PropertyGFATotal', 'LargestPropertyUseTypeGFA',    'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'Age']fig, axes = plt.subplots(3, 3, figsize=(14, 12))axes = axes.flatten()for i, col in enumerate(numeric_cols):    if col in df.columns:        ax = axes[i]        df[col].dropna().hist(bins=50, ax=ax, color='steelblue', edgecolor='white')        ax.set_title(col, fontsize=10)        ax.set_xlabel('')# Masquer les axes videsfor j in range(len(numeric_cols), len(axes)):    axes[j].set_visible(False)plt.suptitle('Distribution des Variables Numériques', fontsize=14, y=1.02)plt.tight_layout()plt.show()

### 8.2 Distribution de la Variable Cible (Émissions CO2)

In [ ]:
# =============================================================================# DISTRIBUTION DES ÉMISSIONS CO2# =============================================================================fig, axes = plt.subplots(1, 2, figsize=(14, 5))# Distribution complèteax1 = axes[0]df['TotalGHGEmissions'].dropna().hist(bins=50, ax=ax1, color='coral', edgecolor='white')ax1.set_title('Distribution des Émissions CO2 (complète)')ax1.set_xlabel('TotalGHGEmissions')ax1.set_ylabel('Fréquence')# Distribution sans les valeurs extrêmesax2 = axes[1]emissions_filtered = df.loc[df['TotalGHGEmissions'] < 1000, 'TotalGHGEmissions']emissions_filtered.hist(bins=50, ax=ax2, color='coral', edgecolor='white')ax2.set_title('Distribution des Émissions CO2 (< 1000)')ax2.set_xlabel('TotalGHGEmissions')ax2.set_ylabel('Fréquence')plt.tight_layout()plt.show()print(f"📊 Statistiques des émissions CO2:")print(df['TotalGHGEmissions'].describe())

---## 9. 🔍 Analyse des Valeurs Manquantes

In [ ]:
# =============================================================================# VALEURS MANQUANTES# =============================================================================missing = df.isnull().sum()missing_pct = (missing / len(df) * 100).round(2)missing_df = pd.DataFrame({    'Missing Count': missing,    'Missing %': missing_pct}).sort_values('Missing Count', ascending=False)# Afficher seulement les colonnes avec des valeurs manquantesmissing_df = missing_df[missing_df['Missing Count'] > 0]print("📊 Colonnes avec valeurs manquantes:\n")print(missing_df)# Visualisationif len(missing_df) > 0:    fig, ax = plt.subplots(figsize=(10, 6))    missing_df['Missing %'].plot(kind='barh', ax=ax, color='tomato')    ax.set_xlabel('Pourcentage manquant (%)')    ax.set_title('Valeurs Manquantes par Colonne')    plt.tight_layout()    plt.show()

---## 10. 🔧 Imputation des Valeurs ManquantesUtilisation de **IterativeImputer (MICE)** pour imputer les valeurs manquantes de manière plus sophistiquée qu'une simple moyenne.

In [ ]:
# =============================================================================# IMPUTATION AVEC ITERATIVE IMPUTER (MICE)# =============================================================================# Sauvegarder l'indexoriginal_index = df.index# Imputationimputer = IterativeImputer(max_iter=10, random_state=42, verbose=0)df_imputed = pd.DataFrame(    imputer.fit_transform(df),    columns=df.columns,    index=original_index)# Correction des valeurs négatives générées par l'imputationdf_imputed[df_imputed < 0] = 0# Vérificationprint("✅ Imputation terminée")print(f"   Valeurs manquantes restantes: {df_imputed.isnull().sum().sum()}")

---## 11. 💾 Sauvegarde des Données Nettoyées

In [ ]:
# =============================================================================# EXPORT DES DONNÉES# =============================================================================# Sauvegardedf_imputed.to_csv(OUTPUT_PATH)print(f"✅ Données sauvegardées dans: {OUTPUT_PATH}")print(f"   Dimensions finales: {df_imputed.shape}")print(f"   Colonnes: {list(df_imputed.columns)}")

---## 12. 📋 Résumé### Transformations Appliquées| Étape | Description | Impact ||-------|-------------|--------|| Filtrage | Exclusion des bâtiments multifamiliaux | Focus sur non-résidentiel || Feature Engineering | Création de Age, ratios, pourcentages | +6 features || Nettoyage | Suppression outliers et valeurs aberrantes | Données plus fiables || Encodage | One-Hot pour variables catégorielles | ML-ready || Imputation | IterativeImputer (MICE) | Pas de valeurs manquantes |### Prochaine Étape➡️ **02_prediction_energy.ipynb** : Modélisation de la consommation énergétique

In [ ]:
# =============================================================================# APERÇU FINAL# =============================================================================print("📊 Aperçu du dataset nettoyé:\n")df_imputed.head()